In [3]:
# 安裝 geckodriver，這是 Firefox WebDriver 的驅動程式。
!apt-get update
!apt install firefox-geckodriver

# 安裝 Selenium 套件
%pip install selenium

from selenium import webdriver
from selenium.webdriver.common.by import By
import pickle
import time
from bs4 import BeautifulSoup
import random
# 前往特定頁面，點擊 follow 按鈕_test
from selenium import webdriver
# 用於生成複雜的使用者交互，比如滑鼠移動、點擊、拖放等
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

zsh:1: command not found: apt-get
The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# 登入 IG

# 初始化 Firefox WebDriver
options = webdriver.FirefoxOptions()
# options.add_argument('--headless')  # 選擇無頭模式，不顯示瀏覽器窗口

driver = webdriver.Firefox(options=options)

# 嘗試加載保存的 cookie
try:
    # 加載保存的 cookie
    driver.get('https://www.instagram.com/accounts/login/')
    cookies = pickle.load(open("instagram_cookies.pkl", "rb"))
    for cookie in cookies:
        driver.add_cookie(cookie)

    # 刷新頁面或前往目標頁面
    driver.refresh()  # 刷新頁面

    # 等待頁面加載完成
    time.sleep(random.uniform(3, 5))

    # 確認是否成功保持登入狀態
    current_url = driver.current_url
    print("當前頁面 URL:", current_url)

except (FileNotFoundError, pickle.UnpicklingError):
    # 如果找不到保存的 cookie 文件，或者加載 cookie 出錯，則需要重新登入
    print("找不到保存的 cookie 文件，或者加載 cookie 出錯，需要重新登入。")

    # 前往 Instagram 登入頁面
    driver.get('https://www.instagram.com/accounts/login/')

    # 等待登入頁面加載
    time.sleep(5)

    # 找到帳號和密碼的輸入框，並輸入登入資訊
    username_input = driver.find_element(By.NAME, 'username')
    password_input = driver.find_element(By.NAME, 'password')

    # 輸入帳號和密碼
    username = 'your_username'
    password = 'your_password'

    username_input.send_keys(username)
    password_input.send_keys(password)

    # 找到登入按鈕，並模擬點擊
    login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    login_button.click()

    # 等待登入完成
    time.sleep(10)  # 等待登入完成，這裡的等待時間可能需要根據網速和登入速度調整

    # 獲取當前頁面的 URL，用於確認是否成功登入
    current_url = driver.current_url
    if 'instagram.com/accounts/login/' not in current_url:
        # 登入成功，獲取並保存 cookie
        pickle.dump(driver.get_cookies(), open("instagram_cookies.pkl", "wb"))
        print("成功登入並保存 cookie。")

    else:
        print("登入失敗或需要進一步操作，無法保存 cookie。")



當前頁面 URL: https://www.instagram.com/


In [6]:


if 'instagram.com/accounts/login/' not in driver.current_url:
    # 登入成功，前往目標頁面
    target_url = 'https://www.instagram.com/group_buy_note/'
    driver.get(target_url)
else:
    print("登入失敗或需要進一步操作，無法前往目標頁面。")

# 等待目標頁面加載完成
time.sleep(random.uniform(3, 5))

# 獲取當前頁面的 URL，用於確認是否成功進入目標頁面
current_url = driver.current_url
print("當前頁面 URL:", current_url)

# 等待 "Following" 按鈕出現
try:
    # 使用 XPath 表達式來尋找 href 包含 /following/ 的 <a> 標籤
    following_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//a[contains(@href, "/following/")]'))
    )
    print("找到了 Following 按鈕")

    # 使用 ActionChains 點擊按鈕
    actions = ActionChains(driver)
    actions.move_to_element(following_button).click().perform()
    print("點擊了 Following 按鈕")

    # 可選：等待 Following 頁面加載完成
    time.sleep(random.uniform(3, 5))
except Exception as e:
    print("找不到 Following 按鈕或點擊失敗:", str(e))

當前頁面 URL: https://www.instagram.com/group_buy_note/
找到了 Following 按鈕
點擊了 Following 按鈕


In [7]:
# 可滾動到最底部！！！

from selenium.webdriver.common.by import By
import time
import random

# 定位特定區域的元素（使用提供的 class 名稱）
class_name = '_aano'
target_element = driver.find_element(By.CLASS_NAME, class_name)

# 初始滾動高度和前一次滾動高度
last_height = 0
scroll_height = 0

while True:
    # 滾動到目前的滾動位置 + 1000 的高度，設置平滑滾動效果
    driver.execute_script('arguments[0].scrollTo({ top: arguments[1], behavior: "smooth" });', target_element, scroll_height + 1000)
    
    # 等待一段時間，讓頁面加載完成
    time.sleep(random.uniform(2, 6))
    
    # 獲取目前的滾動高度
    scroll_height = driver.execute_script('return arguments[0].scrollTop;', target_element)
    print("目前的滾動高度:", scroll_height)
    
    # 獲取頁面的高度
    current_height = driver.execute_script('return arguments[0].scrollHeight;', target_element)
    # print("頁面的高度:", current_height)

    # 判斷是否已經到達頁面底部或無法再滾動
    if scroll_height == last_height:
        print("已無法再滾動，滾動結束。")
        break
    
    # 更新上一次滾動的高度
    last_height = scroll_height
    # print("上一次滾動的高度:", last_height)
    print("-------------------")


目前的滾動高度: 517
頁面的高度: 1506
上一次滾動的高度: 517
-------------------
目前的滾動高度: 1237
頁面的高度: 2226
上一次滾動的高度: 1237
-------------------
目前的滾動高度: 1957
頁面的高度: 2946
上一次滾動的高度: 1957
-------------------
目前的滾動高度: 2677
頁面的高度: 3666
上一次滾動的高度: 2677
-------------------
目前的滾動高度: 3397
頁面的高度: 4386
上一次滾動的高度: 3397
-------------------
目前的滾動高度: 4117
頁面的高度: 5106
上一次滾動的高度: 4117
-------------------
目前的滾動高度: 4837
頁面的高度: 5826
上一次滾動的高度: 4837
-------------------
目前的滾動高度: 5557
頁面的高度: 6546
上一次滾動的高度: 5557
-------------------
目前的滾動高度: 6277
頁面的高度: 7266
上一次滾動的高度: 6277
-------------------
目前的滾動高度: 6997
頁面的高度: 7860
上一次滾動的高度: 6997
-------------------
目前的滾動高度: 7591
頁面的高度: 7860
上一次滾動的高度: 7591
-------------------
目前的滾動高度: 7591
頁面的高度: 7860
已無法再滾動，滾動結束。


In [8]:



# 獲取 Instagram 追蹤者列表

# 獲取頁面源碼
page_source = driver.page_source
# print(page_source)

# 使用 BeautifulSoup 解析頁面源碼
soup = BeautifulSoup(page_source, 'html.parser')
# print(soup)

# 假設 followers_list 已經被正確地提取出來
followers_list = soup.find_all('div', class_='x1dm5mii')

# 創建列表來存儲數據
data = []

for follower in followers_list:
    # 找到追蹤者的帳號名稱
    username_tag = follower.find('a', class_='x1i10hfl')
    if username_tag:
        username = username_tag['href'].strip('/')
    
    # 找到追蹤者的全名
    full_name_tag = follower.find('span', class_='x1lliihq x193iq5w x6ikm8r x10wlt62 xlyipyv xuxw1ft') # group_buy_note
    if full_name_tag:
        full_name = full_name_tag.text
    
    # 打印帳號名稱和全名
    print(f"Username: {username}")
    print(f"Full Name: {full_name}")

    # 建立追蹤者帳號連結
    profile_link = f'https://www.instagram.com/{username}/'
    print(f"Profile Link: {profile_link}")
    print("----------------------------")
    # 添加到數據列表
    data.append([username, full_name, profile_link])

Username: annieyouwant
Full Name: 𝔸𝕟𝕟𝕚𝕖 𝕃𝕚𝕒𝕠♡
Profile Link: https://www.instagram.com/annieyouwant/
----------------------------
Username: ritakuo1110
Full Name: 郭思慧
Profile Link: https://www.instagram.com/ritakuo1110/
----------------------------
Username: r993052
Full Name: Nosoon ® 宋蓉婷
Profile Link: https://www.instagram.com/r993052/
----------------------------
Username: love_ritakao
Full Name: 🩵RITA
Profile Link: https://www.instagram.com/love_ritakao/
----------------------------
Username: mahoushoujojo_2.0
Full Name: 魔法少女2.0｜性別．電影．追劇．閱讀
Profile Link: https://www.instagram.com/mahoushoujojo_2.0/
----------------------------
Username: bobo_1019
Full Name: 胡貝寶 🎈BoBo
Profile Link: https://www.instagram.com/bobo_1019/
----------------------------
Username: nioumo
Full Name: Nioumo |小牛
Profile Link: https://www.instagram.com/nioumo/
----------------------------
Username: tei_foodiary
Full Name: 躺躺 小吃貨日記🍰｜分享：團購 超商 日本 台中 高雄
Profile Link: https://www.instagram.com/tei_foodiary/
---------

In [9]:
# 匯出成 CSV 文件

# 創建 DataFrame
df = pd.DataFrame(data, columns=["Username", "Full Name", "Profile Link"])
print(df)

# 匯出成 CSV 文件
df.to_csv('instagram_followers.csv', index=False)

print("Data has been successfully exported to instagram_followers.csv.")

              Username            Full Name  \
0         annieyouwant          𝔸𝕟𝕟𝕚𝕖 𝕃𝕚𝕒𝕠♡   
1          ritakuo1110                  郭思慧   
2              r993052         Nosoon ® 宋蓉婷   
3         love_ritakao                🩵RITA   
4    mahoushoujojo_2.0  魔法少女2.0｜性別．電影．追劇．閱讀   
..                 ...                  ...   
126            __y1ena           Elena Chen   
127       gdan_pudding               G蛋布丁🥚🍮   
128            ec.wife                 電商人妻   
129      fridaycoolshy           Edward Kao   
130        ellieciouss             愛麗 Ellie   

                                     Profile Link  
0         https://www.instagram.com/annieyouwant/  
1          https://www.instagram.com/ritakuo1110/  
2              https://www.instagram.com/r993052/  
3         https://www.instagram.com/love_ritakao/  
4    https://www.instagram.com/mahoushoujojo_2.0/  
..                                            ...  
126            https://www.instagram.com/__y1ena/  
127       https://w

In [77]:
# 測試印出抓到的網頁

from pprint import pprint

# 獲取頁面源碼
page_source = driver.page_source
# print(page_source)

# 使用 BeautifulSoup 解析頁面源碼
soup = BeautifulSoup(page_source, 'html.parser')
# print(soup)

# 假設 followers_list 已經被正確地提取出來
followers_list = soup.find_all('div', class_='x1dm5mii')

# 使用 pprint 打印格式化的 followers_list
pprint(followers_list)

[<div class="x1dm5mii x16mil14 xiojian x1yutycm x1lliihq x193iq5w xh8yej3"><div class="x9f619 xjbqb8w x78zum5 x168nmei x13lgxp2 x5pf9jr xo71vjh x1pi30zi x1swvt13 xwib8y2 x1y1aw1k x1uhb9sk x1plvlek xryxfnj x1c4vz4f x2lah0s xdt5ytf xqjyukv x1qjc9v5 x1oa3qoh x1nhvcw1"><div class="x9f619 x1n2onr6 x1ja2u2z x1qjc9v5 x78zum5 xdt5ytf x1iyjqo2 xl56j7k xeuugli"><div class="x9f619 x1n2onr6 x1ja2u2z x78zum5 x2lah0s x1qughib x6s0dn4 xozqiw3 x1q0g3np"><div class="x9f619 x1n2onr6 x1ja2u2z xdt5ytf x2lah0s x193iq5w xeuugli xamitd3 x78zum5"><div class="x9f619 xjbqb8w x78zum5 x168nmei x13lgxp2 x5pf9jr xo71vjh xq8finb x1uhb9sk x1plvlek xryxfnj x1c4vz4f x2lah0s xdt5ytf xqjyukv x1qjc9v5 x1oa3qoh x1nhvcw1"><div class=""><div><div aria-disabled="true" class="_aarf" role="button" tabindex="-1"><canvas class="_aarh" height="108" style="left: -5px; position: absolute; top: -5px; height: 54px; width: 54px;" width="108"></canvas><a class="x1i10hfl x1qjc9v5 xjbqb8w xjqpnuy xa49m3k xqeqjp1 x2hbi6w x13fuv20 xu3j5b3 x